# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
# Cell 0
!pip install tensorflow keras tqdm scikit-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Cell 1
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tqdm import tqdm

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [4]:
# Cell 2

class PositionalEncoding(layers.Layer):
    def __init__(self, max_length, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.max_length = max_length
        self.pos_encoding = self.positional_encoding()
        
    def get_angles(self, pos, i):
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / self.embed_dim)
        return pos * angle_rates
    
    def positional_encoding(self):
        angle_rads = self.get_angles(
            np.arange(self.max_length)[:, np.newaxis],
            np.arange(self.embed_dim)[np.newaxis, :]
        )
        
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        
        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)
    
    def call(self, inputs, **kwargs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "max_length": self.max_length,
            "embed_dim": self.embed_dim,
        })
        return config

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, inputs, training=None, **kwargs):
        # Self-attention block
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        
        # Feed-forward network
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.attention.key_dim,
            "num_heads": self.attention.num_heads,
            "ff_dim": self.ffn.layers[0].units,
            "dropout_rate": self.dropout1.rate,
        })
        return config

In [7]:
# Cell 3

def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        words = file.read().splitlines()
    return words

def simulate_missing_letters(word, p_missing=0.4):
    simulated_word = ''
    for char in word:
        simulated_word += '0' if random.random() < p_missing else char
    return simulated_word

def preprocess_data(words, char_to_int, max_word_length, p_missing=0.4):
    sequences = []
    targets = []
    for word in tqdm(words, desc="Preprocessing data"):
        word_with_missing = simulate_missing_letters(word, p_missing)
        sequences.append([char_to_int.get(char, char_to_int['0']) for char in word_with_missing])
        targets.append([char_to_int.get(char, char_to_int['0']) for char in word])
    
    sequences = pad_sequences(sequences, maxlen=max_word_length, padding='post')
    targets = pad_sequences(targets, maxlen=max_word_length, padding='post')
    return sequences, targets

In [9]:
# Cell 4

def build_bert_model(max_word_length, vocab_size, embed_dim, num_heads, ff_dim, num_layers, dropout_rate, num_classes):
    inputs = layers.Input(shape=(max_word_length,))
    
    # Embedding layer
    x = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
    
    # Positional Encoding
    x = PositionalEncoding(max_word_length, embed_dim)(x)
    
    # Transformer Encoder Blocks
    for _ in range(num_layers):
        x = TransformerEncoder(embed_dim, num_heads, ff_dim, dropout_rate)(x)
    
    # Final prediction layer
    outputs = layers.TimeDistributed(layers.Dense(num_classes, activation="softmax"))(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [11]:
# Model parameters
max_word_length = 32
embed_dim = 128
num_heads = 4
ff_dim = 512  # Feed-forward network dimension
num_layers = 4  # Number of Transformer encoder layers
dropout_rate = 0.0

# Load and process data
file_path = '/Users/mazin/Desktop/words_250000_train.txt'
words = load_data(file_path)
chars = sorted(list(set(''.join(words) + '0')))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for c, i in char_to_int.items()}

print("Preprocessing data...")
X, y = preprocess_data(words, char_to_int, max_word_length)
y = to_categorical(y, num_classes=len(chars))

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build BERT-like model
vocab_size = len(chars)
num_classes = len(chars)
model = build_bert_model(
    max_word_length=max_word_length,
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    num_heads=num_heads,
    ff_dim=ff_dim,
    num_layers=num_layers,
    dropout_rate=dropout_rate,
    num_classes=num_classes
)

# Compile with gradient clipping
optimizer = keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Preprocessing data...


Preprocessing data: 100%|███████████| 227300/227300 [00:00<00:00, 470203.25it/s]


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 32, 128)        │         3,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_encoding             │ (None, 32, 128)        │             0 │
│ (PositionalEncoding)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, 32, 128)        │       396,032 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_1           │ (None, 32, 128)        │       396,032 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_2           │ (None, 32, 128)        │       396,032 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_3           │ (None, 32, 128)        │       396,032 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 32, 27)         │         3,483 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,591,067 (6.07 MB)

 Trainable params: 1,591,067 (6.07 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Cell 6: Training Callbacks and Model Training

# Define Callbacks
checkpoint = ModelCheckpoint(
    '/Users/mazin/Desktop/BERT_model/bert_model_checkpoint.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

tensorboard = TensorBoard(
    log_dir='./logs',
    histogram_freq=1,
    write_graph=True,
    update_freq='epoch'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=100,
    min_lr=0.00001,
    verbose=1
)

# Train Model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=128,
    callbacks=[checkpoint, tensorboard, reduce_lr]
)


Epoch 1/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step - accuracy: 0.8813 - loss: 0.4472
Epoch 1: val_loss improved from inf to 0.30423, saving model to /Users/mazin/Desktop/BERT_model/bert_model_checkpoint.keras
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 558s 390ms/step - accuracy: 0.8814 - loss: 0.4471 - val_accuracy: 0.9109 - val_loss: 0.3042 - learning_rate: 0.0010
Epoch 2/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step - accuracy: 0.9121 - loss: 0.3005
Epoch 2: val_loss improved from 0.30423 to 0.29180, saving model to /Users/mazin/Desktop/BERT_model/bert_model_checkpoint.keras
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 551s 387ms/step - accuracy: 0.9121 - loss: 0.3005 - val_accuracy: 0.9144 - val_loss: 0.2918 - learning_rate: 0.0010
Epoch 3/10
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 0.9150 - loss: 0.2895
Epoch 3: val_loss improved from 0.29180 to 0.28579, saving model to /Users/mazin/Desktop/BERT_model/bert_model_checkpoint.keras
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 554s 390ms/step - accurac

In [14]:
# Cell 7
# Save model and mappings
model.save('/Users/mazin/Desktop/BERT_model/bert_model.h5')
np.save('/Users/mazin/Desktop/BERT_model/char_to_int.npy', char_to_int)

In [15]:
# Cell 8
from tensorflow import keras


custom_objects = {
    'PositionalEncoding': PositionalEncoding,
    'TransformerEncoder': TransformerEncoder
}

# Load the model with custom_objects
model = keras.models.load_model(
    '/Users/mazin/Desktop/BERT_model/bert_model.h5',
    custom_objects=custom_objects
)

# Load the character mapping
char_to_int = np.load('/Users/mazin/Desktop/BERT_model/char_to_int.npy', allow_pickle=True).item()

# Create reverse mapping
int_to_char = {i: c for c, i in char_to_int.items()}


In [16]:
# Cell 9
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length, guessed_letters):
    print("BERT-like Model")
    word_encoded = [char_to_int.get(char, char_to_int['0']) for char in word_with_missing]
    word_padded = pad_sequences([word_encoded], maxlen=max_word_length, padding='post')
    
    prediction = model.predict(word_padded)
    predict_vector = prediction[0]
    
    for i, char in enumerate(word_with_missing):
        if char == '0':
            probabilities = predict_vector[i]
            sorted_indices = np.argsort(-probabilities)
            for idx in sorted_indices:
                predicted_char = int_to_char[idx]
                if predicted_char != '0' and predicted_char not in guessed_letters:
                    return predicted_char
    return None  # Return None if all characters are guessed or no suitable character is found

word_with_missing = "an000"
guessed_letters = set(['e','o','i','s','t','d','a'])
predicted_char = predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length, guessed_letters)
print(predicted_char)

BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
c


In [17]:
# Cell 10
class Node:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class DAWG:
    def __init__(self):
        self.root = Node()

    def add_word(self, word):
        current = self.root
        for letter in word:
            if letter not in current.children:
                current.children[letter] = Node()
            current = current.children[letter]
        current.is_end_of_word = True
    
    # search for words with similar pattern using dfs
    def generate_possible_words(self, node, partial_word):
        possible_words = []

        def dfs(current_node, current_prefix):
            if current_node.is_end_of_word:
                possible_words.append(current_prefix)
            if len(current_prefix) < len(partial_word):
                next_letter_index = len(current_prefix)
                next_letter = partial_word[next_letter_index]
                for letter, child_node in current_node.children.items():
                    if next_letter == '_' or next_letter == letter:
                        dfs(child_node, current_prefix + letter)

        dfs(node, '')
        return possible_words

    def predict_next_letter(self, partial_word, guessed_letters):
        word = partial_word.replace(" ", "")
        word = ''.join([letter if letter in guessed_letters else '_' for letter in word])
        # Switch to BERT-like model if 70% of the word has been predicted
        if word.count('_') <= 0.3*(len(word)):
            word_with_missing = word.replace('_', '0')
            return predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length, guessed_letters)
        
        # else use the DAWG data structure to predict the letter using letter frequency
        possible_words = self.generate_possible_words(self.root, partial_word)

        letter_frequency = {}
        for word in possible_words:
            for letter in word:
                if letter not in letter_frequency and letter not in guessed_letters:
                    letter_frequency[letter] = 1
                elif letter not in guessed_letters:
                    letter_frequency[letter] += 1

        sorted_letters = sorted(letter_frequency.items(), key=lambda x: x[1], reverse=True)
        for letter, _ in sorted_letters:
            if letter not in partial_word:
                return letter
            
        # If no suitable letter is found, return a random letter not in guessed_letters
        word = partial_word.replace(" ", "")
        word = ''.join([letter if letter in guessed_letters else '_' for letter in word])
        word_with_missing = word.replace('_', '0')
        return predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length, guessed_letters)

hangman_dawg = DAWG()
for word in words:
    hangman_dawg.add_word(word)

### HangmanAPI class and guess function

In [19]:
# Cell 11
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = '/Users/mazin/Desktop/words_250000_train.txt'
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):
        
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        clean_word = word[::2]
        
        # find length of passed word
        len_word = len(clean_word)

        partial_word = clean_word
        next_letter = hangman_dawg.predict_next_letter(partial_word, self.guessed_letters)
        
        guess_letter = next_letter         
        
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [22]:
api = HangmanAPI(access_token="c919574795aec7964fbc3f2bcf1687", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [24]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() 
practice_success_rate = total_practice_successes / total_practice_runs if total_practice_runs > 0 else 0
print('run %d practice games out of 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

Successfully start a new game! Game ID: 89b7f408bb9e. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '89b7f408bb9e', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e '}
Guessing letter: a
Sever response: {'game_id': '89b7f408bb9e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ e '}
Guessing letter: i
Sever response: {'game_id': '89b7f408bb9e', 'status': 'ongoing', 'tries_remains': 5, 'word': 'i _ i _ _ _ _ e '}
Guessing letter: s
Sever response: {'game_id': '89b7f408bb9e', 'status': 'ongoing', 'tries_remains': 5, 'word': 'i _ i s _ _ _ e '}
Guessing letter: r
Sever response: {'game_id': '89b7f408bb9e', 'status': 'ongoing', 'tries_remains': 5, 'word': 'i r i s _ _ _ e '}
Guessing letter: h
Sever response: {'game_id': '89b7f408bb9e', 'status': 'ongoing', 'tries_remains': 4, 'word': 'i r i s _ _ _ e '}
Guessing letter: t
Sever response: {'game_id': '89b7f408bb9e', 'status': 'ongoing', 'tries_remains': 3, '

In [ ]:
# Initialize counters for this batch only
batch_games = 0
batch_successes = 0

# Run 1000 games
for i in range(1000):
    # Play one game
    success = api.start_game(practice=1, verbose=False)  # Set verbose=False to avoid printing each game
    
    # Update counters
    batch_games += 1
    if success:
        batch_successes += 1
    
    # Print progress every 100 games
    if (i + 1) % 100 == 0:
        current_success_rate = batch_successes / batch_games
        print(f'Completed {batch_games} games. Current success rate = {current_success_rate:.3f}')
    
    # Add a delay between games to avoid too frequent requests
    time.sleep(0.5)  # Adjust the delay as needed to stay within the server's acceptable request rate

# Print final results
final_success_rate = batch_successes / batch_games
print(f'\nFinal Results:')
print(f'Total Games: {batch_games}')
print(f'Total Successes: {batch_successes}')
print(f'Final Success Rate: {final_success_rate:.3f}')


BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
BERT-like Model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
BERT-like Model
1/1 ━━━━━━━━

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status()
success_rate = total_recorded_successes/total_recorded_runs if total_recorded_runs>0 else 0
print('overall success rate = %.3f' % success_rate)
